**Library import**

In [2]:
# Active l'affichage dynamique sur le notebook
import os
import pandas as pd
import matplotlib.pyplot as plt
import math as m
import matplotlib.animation as animation
import numpy as np
%matplotlib qt 
import csv
import os
import imageio


In [88]:

# check all files in the directory
all_files_txt = os.listdir('data/txt/18-05-2024_data_acquisition/')
for filename in all_files_txt:
    if filename.endswith(".txt"):
        with open('data/txt/18-05-2024_data_acquisition/'+filename, 'r') as file:
            lines = file.readlines()

        # Process data
        data = []
        for line in lines:
            if line.strip() and not line.startswith('*'):
                parts = line.strip().split()
                tracker = parts[0].strip(':')
                id = parts[1]
                x = parts[2]
                y = parts[3]
                z = parts[4]
                timestamp = parts[5]
                data.append([tracker, id, x, y, z, timestamp])

        # Write data to a CSV file
        with open('data/csv/18-05-2024/'+filename.replace('.txt', '.csv'), 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['tracker', 'id', 'x', 'y', 'z', 'timestamp'])
            writer.writerows(data)

        print("Data exported in CSV")

Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV
Data exported in CSV


Imported data

In [3]:
# Read all csv files in the directory with panda
dataframes = []
for filename in os.listdir('data/csv/18-05-2024/'):
    if filename.endswith(".csv"):
        df = pd.read_csv('data/csv/18-05-2024/'+filename)
        # add a column with the filename
        df['filename'] = filename
        dataframes.append(df)

Animation at time constant : 200ms

In [6]:
output_dir = 'data/gif/18-05-2024/'
for df in dataframes:
    data_filtered = df[df['tracker'] == 'Filtered'].reset_index(drop=True)
    data_raweee = df[df['tracker'] == 'Raweee'].reset_index(drop=True)

    # Initialize the plot
    fig, ax = plt.subplots()
    line_raw, = ax.plot([], [], 'ro-', label='Raw Data', alpha=0.5)
    line_ema, = ax.plot([], [], 'bo-', label='EMA', alpha=0.5)

    # Set plot limits
    ax.set_xlim(min(data_raweee['x']) - 1, max(data_raweee['x']) + 1)
    ax.set_ylim(min(data_filtered['y']) - 1, max(data_filtered['y']) + 1)

    ax.legend()

    # Initialization function
    def init():
        line_raw.set_data([], [])
        line_ema.set_data([], [])
        return line_raw, line_ema

    # Animation function
    def animate(i):
        x_raw = data_raweee['x'][:i]
        y_raw = data_raweee['y'][:i]
        x_ema = data_filtered['x'][:i]
        y_ema = data_filtered['y'][:i]
        line_raw.set_data(x_raw, y_raw)
        line_ema.set_data(x_ema, y_ema)
        return line_raw, line_ema

    # Create the animation
    ani = animation.FuncAnimation(fig, animate, init_func=init, frames=data_raweee.shape[0], interval=200, blit=True)

    # Show the plot with animation
    plt.xlim(-1, 8)
    plt.ylim(-1, 6)
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Raw Data and EMA Plot')

    plt.close()

    # Save the animation as a GIF
    output_file = output_dir + df['filename'][0].replace('.csv', '.gif')
    ani.save(output_file, writer='pillow', fps=5)

Traceback (most recent call last):
  File "c:\Users\Quent\AppData\Local\Programs\Python\Python312\Lib\site-packages\matplotlib\cbook.py", line 298, in process
    func(*args, **kwargs)
  File "c:\Users\Quent\AppData\Local\Programs\Python\Python312\Lib\site-packages\matplotlib\animation.py", line 916, in _start
    self.event_source.add_callback(self._step)
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'add_callback'


Animation at real time

In [6]:
# Function to calculate time intervals in seconds
def calculate_intervals(timestamps):
    intervals = []
    for i in range(1, len(timestamps)):
        time_diff = (timestamps[i] - timestamps[i-1]) / pd.Timedelta(seconds=1)  # Convert to seconds
        intervals.append(time_diff)
    if len(intervals) > 0:
        intervals.append(intervals.pop(0))  # Move the first interval to the end
    else:
        intervals = [0]  # In case there's only one timestamp
    return intervals

# Ensure output directory exists
output_dir = 'data/gif/18-05-2024/'
os.makedirs(output_dir, exist_ok=True)

# Assume dataframes is a list of your dataframes
for df in dataframes[6:7]:
    data_filtered = df[df['tracker'] == 'Filtered'].reset_index(drop=True)
    data_raweee = df[df['tracker'] == 'Raweee'].reset_index(drop=True)
    
    # Merge data on timestamps to synchronize points
    merged_data = pd.merge_asof(data_raweee.sort_values('timestamp'), data_filtered.sort_values('timestamp'), on='timestamp', suffixes=('_raweee', '_filtered'))

    # Calculate intervals based on the merged timestamps
    combined_timestamps = merged_data['timestamp']
    intervals = calculate_intervals(combined_timestamps)
    print(f"Intervals: {intervals}")

    # Add 0.2 seconds to the last interval
    intervals.append(0.2)

    # Initialize the plot
    fig, ax = plt.subplots()
    ax.set_xlim(-2, 7.5)
    ax.set_ylim(-2, 7.5)
    ax.set_title('Raw vs Filtered Data Plot')
    ax.set_xlabel('X Axis')
    ax.set_ylabel('Y Axis')

    # Create frames
    images = []
    for i in range(len(merged_data)):
        x_raw = merged_data.iloc[i]['x_raweee'] if 'x_raweee' in merged_data.columns else None
        y_raw = merged_data.iloc[i]['y_raweee'] if 'y_raweee' in merged_data.columns else None
        x_fil = merged_data.iloc[i]['x_filtered'] if 'x_filtered' in merged_data.columns else None
        y_fil = merged_data.iloc[i]['y_filtered'] if 'y_filtered' in merged_data.columns else None
        
        if x_raw is not None and y_raw is not None:
            plt.scatter(x_raw, y_raw, color='red', s=30, alpha=0.5, label='Raw' if i == 0 else "")
        if x_fil is not None and y_fil is not None:
            plt.scatter(x_fil, y_fil, color='blue', s=30, alpha=0.5, label='Filtered' if i == 0 else "")

        # Add legend on the first iteration
        if i == 0:
            ax.legend()

        # Pause for the animation based on time difference
        plt.pause(intervals[i])
    
    plt.show()
    plt.close()

MergeError: Incompatible merge dtype, dtype('O') and dtype('O'), both sides must have numeric dtype